In [39]:
import pickle
from tqdm import tqdm
import pandas as pd
import json


import os 

os.chdir('c:/Users/Domen/IronHack/01_projects/IH_final_project_data-jobs')
print(os.getcwd())

from src.myModules import my_NLP as myNLP

%load_ext autoreload
%autoreload 2

c:\Users\Domen\IronHack\01_projects\IH_final_project_data-jobs
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
print(os.getcwd())
with open("config/config.json", 'r') as f:
    config = json.load(f)
    
NLP_data_path = config['NLP_data_path']

print(NLP_data_path)


nlp_df_input = 'NLP_title'

c:\Users\Domen\IronHack\01_projects\IH_final_project_data-jobs
data/NLP_data/


## Test Code - NLP

load cleaned dataframe

In [41]:
with open(f"{NLP_data_path}{nlp_df_input}.pkl", "rb") as file:
    df = pickle.load(file)


In [42]:
df.columns

Index(['city', 'company', 'cp1_title', 'cp2_title', 'employment_type',
       'first_match', 'id', 'industries', 'job_description', 'job_function',
       'job_level', 'keyword', 'language', 'new_job_title', 'posting_date',
       'scraping_date', 'seniority_level', 'title', 'url'],
      dtype='object')

In [43]:
print(df.columns)
nlp_df = df[df['first_match']==True]

nlp_df = nlp_df[[ 'id', 'company', 'title','new_job_title', 'job_description',  'language', 'url']]

Index(['city', 'company', 'cp1_title', 'cp2_title', 'employment_type',
       'first_match', 'id', 'industries', 'job_description', 'job_function',
       'job_level', 'keyword', 'language', 'new_job_title', 'posting_date',
       'scraping_date', 'seniority_level', 'title', 'url'],
      dtype='object')


In [44]:
keywords = ['data_analyst', 'analytics_engineer','bi_analyst', 'ml_engineer', 'business_analyst', 'business_intelligence_analyst', 'data_scientist']

In [45]:
nlp_df = nlp_df[nlp_df['new_job_title'].apply(lambda job_title: any(skill in keywords for skill in job_title))]

In [46]:
nlp_df.fillna("", inplace=True)

#### Language detection

In [47]:
tricky_re_skills = ['r', 'scala', 'sas', 'gcp']

In [48]:
skills = ['python', 'sql', 'gitlab',' r ','bi-bool','r shiny', 'power bi', 'visual analytics' 'cognos bi','aws', 'spark', 'azure', 'tableau', 'java', 'excel', 'hadoop', 'scala', 
           'snowflake', 'kafka', 'nosql', 'databricks', ' git ', 'redshift', 'airflow', 'oracle', 'sap', 'ad-hoc', 
           'sql server', 'docker', 'kubernetes', 'power_bi', 'c++', 'numpy', 'dplyr', 'matplotlib', 
           'seaborn', 'ggplot', 'hypothesis testing', 'regression analysis', 'predictive analysis', 'bayesian statistics', 
           'scikit', 'tensorflow', 'pytorch', 'xgboost', 'deep learning', 'hadoop', 'bigquery', 'airflow', 'nltk', 'spacy', 
           'nlp', 'aws', 'azure', 'looker', 'ms-office', ' ml ', ' nlp ', ' pandas', 'e2e']

First round of skill extraction from text

In [49]:
tqdm.pandas(desc="Processing")
nlp_df['nlp_job_description'] = nlp_df['job_description'].progress_apply(lambda x: x.lower().strip()) # lower job description text
nlp_df = nlp_df.progress_apply(lambda row: myNLP.check_skills(row, skills), axis=1) # use check_skill function to extract skills from text which matches words in skills list




Processing: 100%|██████████| 1687/1687 [00:00<00:00, 40156.58it/s]


















































































Processing: 100%|██████████| 1687/1687 [00:04<00:00, 380.44it/s]


In [50]:
nlp_df.head(2)

,company,id,job_description,language,new_job_title,nlp_job_description,skills,title,url
1,Almedia,3719620327,Almedia helps companies grow by promoting thei...,en,"[data_analyst, bi_analyst]",almedia helps companies grow by promoting thei...,"[SQL, SQL, Python, Excel]",Data Analyst - Business Intelligence,https://www.linkedin.com/jobs/search?keywords=...
3,Ultramarin,3672508925,About The PositionWe are looking for a top-tie...,en,[data_analyst],about the positionwe are looking for a top-tie...,"[Excel, SQL, Python, Pandas, Matplotlib]",Financial Data Analyst (m/f/d),https://www.linkedin.com/jobs/search?keywords=...


Investigate german and english text to identify further skills to extract 

In [51]:
filtered_view = nlp_df[nlp_df['new_job_title'].apply(lambda job_title: any(skill in ['data_analyst'] for skill in job_title))] # filter dataset on specific job title
filtered_view.head(20)

,company,id,job_description,language,new_job_title,nlp_job_description,skills,title,url
1,Almedia,3719620327,Almedia helps companies grow by promoting thei...,en,"[data_analyst, bi_analyst]",almedia helps companies grow by promoting thei...,"[SQL, SQL, Python, Excel]",Data Analyst - Business Intelligence,https://www.linkedin.com/jobs/search?keywords=...
3,Ultramarin,3672508925,About The PositionWe are looking for a top-tie...,en,[data_analyst],about the positionwe are looking for a top-tie...,"[Excel, SQL, Python, Pandas, Matplotlib]",Financial Data Analyst (m/f/d),https://www.linkedin.com/jobs/search?keywords=...
5,Orange Quarter,3585738180,Orange Quarter are working with an exciting st...,en,[data_analyst],orange quarter are working with an exciting st...,[E2E],Data Analyst,https://www.linkedin.com/jobs/search?keywords=...
6,Orange Quarter,3585734564,Orange Quarter are currently looking for a Dat...,en,[data_analyst],orange quarter are currently looking for a dat...,"[SQL, Python, Snowflake, BigQuery]",Data Analyst,https://www.linkedin.com/jobs/search?keywords=...
8,Orange Quarter,3585734595,We are currently working with an emerging SaaS...,en,[data_analyst],we are currently working with an emerging saas...,"[E2E, SQL, Python, R ]",Data Analyst,https://www.linkedin.com/jobs/search?keywords=...
10,kevin.,3687813018,kevin. is different. While everyone was talkin...,en,[data_analyst],kevin. is different. while everyone was talkin...,"[SQL, Tableau, Power BI, Looker, SAP]",Product Data Analyst,https://www.linkedin.com/jobs/search?keywords=...
14,CBTW,3706371923,Positive Thinking Company ist Teil unseres wel...,de,[data_analyst],positive thinking company ist teil unseres wel...,"[Tableau, Power BI, R ]",Data Analyst (m/w/d),https://www.linkedin.com/jobs/search?keywords=...
17,Qualitas Energy Deutschland GmbH,3701510210,Powering Change.Seit mehr als 16 Jahren setzen...,de,[data_analyst],powering change.seit mehr als 16 jahren setzen...,[Python],Data Analyst (w/m/d),https://www.linkedin.com/jobs/search?keywords=...
18,Talentschmiede Unternehmensberatung AG,3582381366,"As a cross-industry IT management consultancy,...",en,[data_analyst],"as a cross-industry it management consultancy,...","[excel, Oracle, SQL]",Young Professional Client Data Analyst (m/f/d) ←,https://www.linkedin.com/jobs/search?keywords=...
20,Digital Charging Solutions,3725401052,Your missionAs a crucial part of the Business ...,en,[data_analyst],your missionas a crucial part of the business ...,"[scala, SQL, Python, Tableau]",Business Process & Data Analyst (f/m/d),https://www.linkedin.com/jobs/search?keywords=...


In [52]:
df_ger = nlp_df[nlp_df['language'] == 'de']
df_eng = nlp_df[nlp_df['language'] == 'en']

In [53]:
df_ger.head(10)

,company,id,job_description,language,new_job_title,nlp_job_description,skills,title,url
14,CBTW,3706371923,Positive Thinking Company ist Teil unseres wel...,de,[data_analyst],positive thinking company ist teil unseres wel...,"[Tableau, Power BI, R ]",Data Analyst (m/w/d),https://www.linkedin.com/jobs/search?keywords=...
17,Qualitas Energy Deutschland GmbH,3701510210,Powering Change.Seit mehr als 16 Jahren setzen...,de,[data_analyst],powering change.seit mehr als 16 jahren setzen...,[Python],Data Analyst (w/m/d),https://www.linkedin.com/jobs/search?keywords=...
22,Qcells Europe,3720128757,Setzt neue Energie frei - Dein Einsatz:Sie int...,de,[data_analyst],setzt neue energie frei - dein einsatz:sie int...,"[Python, SQL]",Data Analyst / System Developer (m/f/d),https://www.linkedin.com/jobs/search?keywords=...
26,Michael Page,3713068792,Pionier in Grüner Energie|Internationales Unte...,de,[data_analyst],pionier in grüner energie|internationales unte...,"[SQL, Python]",Data Analyst (m/w/d) Klimadaten,https://www.linkedin.com/jobs/search?keywords=...
28,Lloyds Bank GmbH,3710653394,Über eine Million zufriedene Kund*innen konnte...,de,[data_analyst],über eine million zufriedene kund*innen konnte...,"[Ad-hoc, Power BI, SQL]",Data Analyst im Marketing-Bereich (m/w/d),https://www.linkedin.com/jobs/search?keywords=...
31,energicos GmbH,3694113016,BerlinVollzeitab sofortAusschreibung Data Scie...,de,"[data_scientist, data_analyst]",berlinvollzeitab sofortausschreibung data scie...,"[Python, Python]",Data Scientist / Data Analyst (m/w/d),https://www.linkedin.com/jobs/search?keywords=...
34,BWI GmbH,3711964211,Sorgen Sie gemeinsam mit uns für die digitale ...,de,"[data_analyst, controlling_, bi_analyst]",sorgen sie gemeinsam mit uns für die digitale ...,NaN,Data Analyst / Controller Business Intelligenc...,https://www.linkedin.com/jobs/search?keywords=...
35,SCI Verkehr GmbH,3704824751,Analystin im Mobilitätssektor (w/m/d) für die ...,de,"[business_analyst, data_analyst]",analystin im mobilitätssektor (w/m/d) für die ...,[Excel],Business Analyst / Data Analyst Logistik (w/m/...,https://www.linkedin.com/jobs/search?keywords=...
38,Michael Page,3699420235,"Power BI, SQL und Azure als Alltag|Bundesweite...",de,"[cloud_engineer, data_analyst]","power bi, sql und azure als alltag|bundesweite...","[Power BI, SQL, Azure, Power BI, Tableau, SAP,...",Cloud Data Analyst (m/w/d),https://www.linkedin.com/jobs/search?keywords=...
39,Talentschmiede Unternehmensberatung AG,3582374933,Die Talentschmiede fördert als branchenübergre...,de,[data_analyst],die talentschmiede fördert als branchenübergre...,"[Oracle, SQL]",Young Professional Client Data Analyst (m/w/d) ←,https://www.linkedin.com/jobs/search?keywords=...


In [54]:
# df_ger = df_ger.progress_apply(lambda row: myNLP.check_german_style(row), axis=1)
# df_ger.head(2)

In [55]:
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(myNLP.clean_up)
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(myNLP.stem_and_lemmatize_german) # lemmatize strings
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(myNLP.tokenize) # tokenize
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(myNLP.remove_stopwords_german) #remove stopwords from
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(myNLP.re_blob) # reblob
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(lambda x: x.lower().strip())






Processing: 100%|██████████| 1350/1350 [00:00<00:00, 7177.94it/s]
C:\Users\Domen\AppData\Local\Temp\ipykernel_130572\1913035804.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(myNLP.clean_up)

































































































































































































































































































































































































































































































In [ ]:
df_ger['nlp_job_description'].head(10)

In [ ]:
myNLP.word_cloud(df_ger, 'nlp_job_description')

english dataset

In [ ]:
df_eng['nlp_job_description'] = df_eng['job_description'].progress_apply(myNLP.clean_up) # clean dataset
df_eng['nlp_job_description'] = df_eng['job_description'].progress_apply(myNLP.stem_and_lemmatize_german) # lemmatize strings
df_eng['nlp_job_description'] = df_eng['nlp_job_description'].progress_apply(myNLP.tokenize) # tokenize
df_eng['nlp_job_description'] = df_eng['nlp_job_description'].progress_apply(myNLP.remove_stopwords_german) #remove stopwords from
df_eng['nlp_job_description'] = df_eng['nlp_job_description'].progress_apply(myNLP.re_blob) # reblob
df_eng['nlp_job_description'] = df_eng['nlp_job_description'].progress_apply(lambda x: x.lower().strip())

In [ ]:
display(df_eng['nlp_job_description'].head(10))

In [ ]:
myNLP.word_cloud(df_eng, 'nlp_job_description')

### clustering using BERT Model and KMEAN 

In [ ]:
from transformers import BertModel, BertTokenizer
import torch
from sklearn.cluster import KMeans 
import pandas as pd
import numpy as np


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

In [ ]:
def encode_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:,0,:].detach().numpy()
    return embeddings

In [ ]:
embeddings = np.array([encode_text(text) for text in tqdm(nlp_df['job_description'])]).squeeze()


try:
    with open("data/model_data/bert_embedding.pkl", "wb") as file:
        pickle.dump(embeddings, file)
except Exception as e:
    print(f"Error: {e}")

In [ ]:
with open("data/model_data/bert_embedding.pkl", "wb") as file:
    pickle.dump(embeddings, file)